In [1]:
# Install the required libraries (already installed)
!pip3 install -qU datasets
!pip3 install -q langchain_community
!pip3 install -q sentence-transformers
!pip3 install -q langchain_huggingface
!pip3 install -q qdrant_client
!pip3 install numpy==1.26
# Restart your session after install

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 13.3 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2025.3.2 requires fsspec==2025.3.2, but you have fsspec 2025.3.0 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cuda-cupti-cu12 12.5.82 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-nvrtc-cu12==12.4.127; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cuda-nvrtc-cu12 12.5.82 which is incompatible.
torch 2.6.

In [1]:
!pip install cohere

# Naive retrieval
In this section, we implement a naive pipeline for performing retrieval on a user query. Specifically the steps will be:
- Load a Q&A dataset that represent possible user's query
- Choose an embedding model and create a dense representation of the query
- Query our vector store provide the most relevant chunks using cosinge similarity

 This represent the first step of a RAG pipeline where relevant context is retrieved.

## Load dataset of Q&A
Let's load our dataset of Q&A about EO, each sample is composed of a question and an answer

In [2]:
# ! pip install -U cohere
import cohere
from google.colab import userdata

COHERE_API_KEY=userdata.get('cohere-mine-pischool')

co = cohere.ClientV2(
    COHERE_API_KEY
)

In [3]:
from datasets import load_dataset
import random
import json

qa = load_dataset('eve-esa/eve-is-open-ended')['train']
qa2 = load_dataset('eve-esa/hardest-50-qna')['train']

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/306 [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/30.4k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/313 [00:00<?, ? examples/s]

README.md:   0%|          | 0.00/304 [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/8.75k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/50 [00:00<?, ? examples/s]

In [ ]:
# # Take a random idx
# idx = 0

# question = qa2[idx]['Question']
# answer = qa2[idx]['Answer']


# print('Question: ', question)
# print('Answer: ', answer)

### Embeddings

An embeddings model in Retrieval-Augmented Generation (RAG) is a neural network that converts text into dense vector representations (embeddings) in a **high-dimensional space**. These models take text as input and produce a fixed-length array of numbers, a numerical fingerprint of the text's semantic meaning. Embeddings allow search system to find relevant documents not just based on keyword matches, but on semantic understanding.

Embeddings models are trained on large text corpora using unsupervised learning techniques. They learn to encode the semantic meaning of words, sentences, and documents in a way that captures relationships between them. For example, embeddings models can learn that "cat" and "dog" are similar because they are both animals, or that "apple" and "orange" are similar because they are both fruits.

There are many pre-trained embedding models available, each suited to different types of data and use cases. For our application, we use [Indus](https://huggingface.co/papers/2405.10725), a fine-tuned encoder-only transformer model trained specifically on scientific journals and articles related to NASA’s Science Mission Directorate (SMD).

Choosing the right embedding model is a critical step in building an effective retrieval system. Ideally, the embedding model should be trained—or at least fine-tuned—on data similar to the target documents. Since our corpus consists of scientific texts focused on Earth Observation, Indus is a better fit than a general-purpose model, as it captures domain-specific terminology and semantics more accurately.
<figure>

<img src="https://weaviate.io/assets/images/embedding-models-0c04d93c0be28dd63a0e8781c4e8685d.jpg" width='800px'>
<figcaption><a href='https://weaviate.io/blog/how-to-choose-an-embedding-model'>Source</a></figcaption>
<figure>



In [4]:
from langchain_community.embeddings import HuggingFaceEmbeddings

# Load the embeddings model
model_name = "nasa-impact/nasa-smd-ibm-st-v2"
encode_kwargs = {"normalize_embeddings": True}
indus_embd = HuggingFaceEmbeddings(
    model_name=model_name,  encode_kwargs=encode_kwargs
)

<ipython-input-4-3026926332>:6: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  indus_embd = HuggingFaceEmbeddings(


modules.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/4.57k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/54.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/864 [00:00<?, ?B/s]

...

vocab.json:   0%|          | 0.00/817k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/476k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.15M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/280 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/191 [00:00<?, ?B/s]

### Vector Store

Vector stores are specialized databases designed to efficiently index and retrieve information using vector representations of data. Vector stores leverages the dense representation by reducing the task of finding similar documents to a search in a high-dimensional space. This search is made by comparing the vector representation of the **query** with the vector representation of the **documents** in the database. The documents that are closer to the query vector are considered more similar to the query.

Wrapping up the retrieval process is composed of:
- **Documents embedding**
- **Store the embeddings in a VectorStore**
- **Query embedding**
- **Retrieve** the most similar documents to the query


The most popular and simple setup is using the **cosine similarity** to compare the vectors and retrieve the **top k** most similar ones


<figure>
<img src="https://python.langchain.com/assets/images/vectorstores-2540b4bc355b966c99b0f02cfdddb273.png" width="800"/>
<figcaption><a href='https://python.langchain.com/docs/concepts/vectorstores/'>Source</a></figcaption>
</figure>


## Connect to QDrant

We are using Qdrant as our vector store, which has been preloaded with all the relevant documents needed for retrieval. In this section, we will connect to the Qdrant instance and select the specific collection that contains our indexed data. This will enable us to perform efficient semantic searches and support the Retrieval-Augmented Generation (RAG) process used in our Q&A tasks.

In [ ]:
# Examples of retrieval pipeline using the embedding function and the API from QDrant
from qdrant_client import QdrantClient
import os

QDRANT_API_KEY=userdata.get('QDRANT_API_KEY')
QDRANT_URL=userdata.get('QDRANT_URL')

# Enstablish a connection wit the vector store
client = QdrantClient(
    url=QDRANT_URL,
    api_key=QDRANT_API_KEY
)

<ipython-input-10-959728562>:9: UserWarning: Qdrant client version 1.14.2 is incompatible with server version 1.9.4. Major versions should match and minor version difference must not exceed 1. Set check_compatibility=False to skip version check.
  client = QdrantClient(


In [11]:
from langchain_core.documents import Document
from langchain_core.retrievers import BaseRetriever
from typing import List
from qdrant_client import QdrantClient
from pydantic import PrivateAttr, Field
from typing import List, Optional, Dict
from qdrant_client.models import Filter, PointStruct

from langchain_core.callbacks import CallbackManagerForRetrieverRun

import numpy as np

# Let's define our retriever class to have a nice interface
class QdrantRetriever():
    def __init__(self, embedding, api_key, qdrant_url, collection_name='esa-nasa-workshop', k: int = 3):
        self._client = QdrantClient(url=qdrant_url, api_key=api_key)
        self.embedding = embedding
        self.collection_name = collection_name
        self.k = k


    def get_relevant_documents(self, query: str) -> List[Document]:
        query_emb = self.embedding.embed_query(query)

        search_result = self._client.search(
            collection_name=self.collection_name,
            query_vector=query_emb,
            limit=self.k,
        )

        docs = []
        for hit in search_result:
            # Adjust based on your actual data structure
            data = hit.payload
            content = data.get("text", "")
            metadata = {}
            for key, value in data.items():
                if key != "text":
                    metadata[key] = value
            docs.append(Document(page_content=content, metadata=metadata))

        return docs


In [12]:
import json

# Let's define our retriever
retriever = QdrantRetriever(indus_embd, QDRANT_API_KEY, QDRANT_URL, k=3)

<ipython-input-11-3583072626>:16: UserWarning: Qdrant client version 1.14.2 is incompatible with server version 1.9.4. Major versions should match and minor version difference must not exceed 1. Set check_compatibility=False to skip version check.
  self._client = QdrantClient(url=qdrant_url, api_key=api_key)


In [13]:
# Format retrieved documents:
def format_docs(docs):
  doc_str = ''
  for i, doc in enumerate(docs):
    doc_str += f'Document n. {i+1}\n'
    doc_str += f'TITLE: {doc.metadata.get("title", "No title")}\n' # Add title's of the paper
    if type(doc.metadata.get("headers", {})) is str:
      # Parse JSON string
      doc.metadata['headers'] = json.loads(doc.metadata.get("headers", {}))
    for key, value in doc.metadata.get("headers", {}).items():
      doc_str += f'{value}\n'
    doc_str += f'URL: {doc.metadata.get("url", "No url")}\n\n' # Add URL of the paper
    doc_str += f'{doc.page_content}\n\n'
  return doc_str


def format_docs_dict(docs):
    formatted = []
    for doc in docs:
        # Extract title
        title = doc.metadata.get("title", "No title")

        # Extract and parse headers if needed
        headers = doc.metadata.get("headers", {})
        if isinstance(headers, str):
            try:
                headers = json.loads(headers)
            except json.JSONDecodeError:
                headers = {}

        # Create a snippet from headers and page content
        header_snippets = "\n".join(headers.values())
        snippet = f"{header_snippets}\n\n{doc.page_content}".strip()

        # Create the formatted dict
        formatted.append({
            "data": {
                "title": title,
                "snippet": snippet
            }
        })
    return formatted

In [14]:
#question = qa[2]['question']
question = qa2[2]['Question']

docs = retriever.get_relevant_documents(question)

print(format_docs(docs))

messages = [{"role": "user", "content": question}]
response = co.chat(
    model="command-a-03-2025",
    messages=messages,
    documents=format_docs_dict(docs),
)

print(response.message.content[0].text)
print(response.message.citations)

<ipython-input-11-3583072626>:25: DeprecationWarning: `search` method is deprecated and will be removed in the future. Use `query_points` instead.
  search_result = self._client.search(


Document n. 1
TITLE: CLARA-A1: a cloud, albedo, and radiation dataset from 28 yr of global AVHRR data
URL: http://dx.doi.org/10.5194/acp-13-5351-2013

A comprehensive description of CM SAF activities and plans is given by Schultz et al. (2009).\n\nSatellite observations have a very short history in a climatological perspective; useful systematic measurements did not begin until around 1980 (Davis, 2007). Consequently, it is only possible to study satellite-based observational series spanning at most three decades. Despite this seemingly critical limitation, it is important that these observations are analysed and prepared for future continuation in order to be used for careful evaluation of short- and medium-term climate fluctuations, in particular those suggested by climate scenarios from climate model simulations.

Document n. 2
TITLE: GEO-Bench: Toward Foundation Models for Earth Monitoring
GEO-Bench:
Appendix C Societal Impact of Foundation Models for Earth Monitoring
Climate mitig

In [15]:
#question = qa[2]['question']
question = qa2[2]['Question']
gold_answer = qa2[2]['Answer']
results = {}

# -- QUERY EXPANSION METHODS --
expansions = {
    "sub_query": [f"{question} - specific aspect {i}" for i in range(1, 4)],
    "multi_query": [question, f"{question} with examples", f"{question} explained differently"],
    "CoVe": [f"{question} - similar context {i}" for i in range(1, 3)]
}

for method, queries in expansions.items():
    answers = []
    for q in queries:
        docs = retriever.get_relevant_documents(q)
        formatted_docs = format_docs_dict(docs)
        messages = [{"role": "user", "content": q}]
        response = co.chat(model="command-a-03-2025", messages=messages, documents=formatted_docs)
        answers.append({
            "query": q,
            "response": response.message.content[0].text,
            "citations": response.message.citations
        })
    results[f"expansion_{method}"] = answers

# -- QUERY TRANSFORMATION METHODS --
transformations = {
    "rewrite": f"Rewrite the following clearly with better stated keywords for RAG: {question}",
    "reverse_hyde": f"Start with a hypothetical answer and infer the question: {question}",
    "step_back": f"Before answering, explain what the question is asking: {question}"
}

for method, transformed_query in transformations.items():
    docs = retriever.get_relevant_documents(transformed_query)
    formatted_docs = format_docs_dict(docs)
    messages = [{"role": "user", "content": transformed_query}]
    response = co.chat(model="command-a-03-2025", messages=messages, documents=formatted_docs)
    results[f"transform_{method}"] = {
        "query": transformed_query,
        "response": response.message.content[0].text,
        "citations": response.message.citations
    }

# Optional: Print or save for comparison
for method, output in results.items():
    print(f"\n--- {method.upper()} ---")
    if isinstance(output, list):
        for i, item in enumerate(output):
            print(f"\nQuery {i+1}: {item['query']}")
            print(f"Response: {item['response']}")
            print(f"Citations: {item['citations']}")
    else:
        print(f"Query: {output['query']}")
        print(f"Response: {output['response']}")
        print(f"Citations: {output['citations']}")


<ipython-input-11-3583072626>:25: DeprecationWarning: `search` method is deprecated and will be removed in the future. Use `query_points` instead.
  search_result = self._client.search(



--- EXPANSION_SUB_QUERY ---

Query 1: Why does satellite data play a crucial part for monitoring climate change? - specific aspect 1
Response: Working group II of the Intergovernmental Panel on Climate Change (IPCC) (Houghton et al., 2001) used SAT as the main climate change indicator, due to its predominance in the existing literature and its large scientific consistency as such.

Satellite observations have a very short history in a climatological perspective; useful systematic measurements did not begin until around 1980 (Davis, 2007). Consequently, it is only possible to study satellite-based observational series spanning at most three decades. Despite this seemingly critical limitation, it is important that these observations are analysed and prepared for future continuation in order to be used for careful evaluation of short- and medium-term climate fluctuations, in particular those suggested by climate scenarios from climate model simulations.
Citations: [Citation(start=0, end=

In [28]:
judge_prompt = """
You are an expert evaluator. Compare the following generated answers against the provided gold/reference answer.

Your job is to identify the one that best matches the reference in terms of:
- Accuracy
- Completeness
- Clarity
- Use of supporting information

Gold Answer:
{gold_answer}

Generated Answers:
{answers_block}

Please respond in this format:
- Best Method: <method_name>
- Justification: <why it matches the reference best>
"""

answers_block = ""
for method, output in results.items():
    if isinstance(output, list):
        for i, item in enumerate(output):
            answers_block += f"\n[{method} #{i+1}]\nQuery: {item['query']}\nAnswer: {item['response']}\nCitations: {item['citations']}\n"
    else:
        answers_block += f"\n[{method}]\nQuery: {output['query']}\nAnswer: {output['response']}\nCitations: {output['citations']}\n"

judge_input = judge_prompt.format(gold_answer=gold_answer, answers_block=answers_block)

judge_response = co.chat(
    model="command-a-03-2025",
    messages=[{"role": "user", "content": judge_input}]
)

print(judge_response.message.content[0].text)

- Best Method: [expansion_multi_query #1]
- Justification: This method provides a comprehensive and accurate answer that closely aligns with the reference. It explains that satellite data is crucial for monitoring climate change because it offers a wide range of environmental data, including atmospheric composition and surface properties, over large spatial areas. It also addresses the limitation of the short history of satellite observations, which is a key aspect mentioned in the reference. The answer is clear, complete, and supported by relevant citations, making it the best match.


In [25]:
question

'Why does satellite data play a crucial part for monitoring climate change?'

In [27]:
results['expansion_multi_query'][0]['response']

'Satellite data plays a crucial part in monitoring climate change because it provides a plethora of environmental data, including information on atmospheric composition and surface properties over wide spatial areas. Their instruments provide coverage that is spatially and temporally delineated based on orbiting patterns and swath width. Useful systematic measurements did not begin until around 1980, so it is only possible to study satellite-based observational series spanning at most three decades. Despite this seemingly critical limitation, it is important that these observations are analysed and prepared for future continuation in order to be used for careful evaluation of short- and medium-term climate fluctuations, in particular those suggested by climate scenarios from climate model simulations.'